<a href="https://colab.research.google.com/github/tonkatsu7/learnLangChain/blob/main/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright © 2023 Patrick Loeber

https://colab.research.google.com/drive/1VOwJpcZqOXag-ZXi-52ibOx6L5Pw-YJi?usp=sharing#scrollTo=nTDgRy0jKDkP

# LangChain

LangChain is a framework for developing applications powered by language models.

- GitHub: https://github.com/hwchase17/langchain
- Docs: https://python.langchain.com/en/latest/index.html

### Overview:
- Installation
- LLMs
- Prompt Templates
- Chains
- Agents and Tools
- Memory
- Document Loaders
- Indexes

## Installation

In [8]:
pip freeze

aiohttp==3.8.4
aiosignal==1.3.1
altair==4.2.2
anyio==3.6.2
appdirs==1.4.4
appnope==0.1.2
argon2-cffi==20.1.0
asgiref==3.3.4
astroid==2.5.1
async-generator==1.10
async-timeout==4.0.2
attrs==20.3.0
Babel==2.9.0
backcall==0.2.0
backoff==2.2.1
black==21.5b0
bleach==3.3.0
blinker==1.6.2
cachetools==5.3.0
certifi==2020.12.5
cffi==1.14.5
chardet==4.0.0
charset-normalizer==3.1.0
chromadb==0.3.21
click==7.1.2
clickhouse-connect==0.5.22
colorama==0.4.4
dataclasses-json==0.5.7
decorator==4.4.2
defusedxml==0.7.0
distlib==0.3.6
Django==3.2
duckdb==0.7.1
entrypoints==0.3
et-xmlfile==1.1.0
fastapi==0.95.1
filelock==3.12.0
flake8==3.9.1
frozenlist==1.3.3
fsspec==2023.4.0
gitchangelog==3.0.4
gitdb==4.0.10
GitPython==3.1.31
h11==0.14.0
hnswlib==0.7.0
httptools==0.5.0
huggingface-hub==0.14.1
idna==2.10
importlib-metadata==6.6.0
ipykernel==5.5.0
ipython==7.21.0
ipython-genutils==0.2.0
isort==5.7.0
itsdangerous==1.1.0
jedi==0.18.0
Jinja2==3.1.2
joblib==1.2.0
json5==0.9.5
jsonschema==3.2.0
jupyter-client==6

In [ ]:
!pip install langchain

## 1. LLMs

A generic interface for all LLMs. See all LLM providers: https://python.langchain.com/en/latest/modules/models/llms/integrations.html

In [ ]:
!pip install openai

In [2]:
from getpass import getpass
OPENAI_API_KEY = getpass("OpenAI API Key: ")

In [3]:
RG = input("Azure OpenAI resource group name: ")
OPENAI_API_BASE = 'https://' + RG +'.openai.azure.com'

In [4]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

os.environ['OPENAI_API_TYPE'] = 'azure'
# API version to use (Azure has several)
os.environ['OPENAI_API_VERSION'] = '2023-05-01' #'2022-12-01' or '2023-03-15-preview'
# base URL for your Azure OpenAI resource
os.environ['OPENAI_API_BASE'] = OPENAI_API_BASE

In [5]:
# from langchain.llms import OpenAI
from langchain.llms import AzureOpenAI

# llm = OpenAI(temperature=0.9)  # model_name="text-davinci-003"
llm = AzureOpenAI(deployment_name="gpt301",
                  model_name="text-davinci-003",
                  temperature=0.9)

text = "What would be a good company name for a company that makes colorful socks?"
print(llm(text))



BrightToes Socks.


In [ ]:
!pip install huggingface_hub

In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = "YOUR_HF_TOKEN"

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
# https://huggingface.co/google/flan-t5-xl
llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":64})

llm("translate English to German: How old are you?")

## 2. Prompt Templates

LangChain faciliates prompt management and optimization.

Normally when you use an LLM in an application, you are not sending user input directly to the LLM. Instead, you need to take the user input and construct a prompt, and only then send that to the LLM.

In [3]:
llm("Can Barack Obama have a conversation with George Washington?")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: The server had an error while processing your request. Sorry about that!.


'\n\nNo, Barack Obama cannot have a conversation with George Washington since George Washington is no longer living.'

In [4]:
prompt = """Question: Can Barack Obama have a conversation with George Washington?

Let's think step by step.

Answer: """
llm(prompt)

' No, Barack Obama cannot have a conversation with George Washington. George Washington passed away in 1799, over 200 years before Barack Obama was born. Moreover, even if it were possible to talk to someone who had died that long ago, the technology to do so does not exist.'

In [5]:
from langchain import PromptTemplate

template = """Question: {question}

Let's think step by step.

Answer: """

prompt = PromptTemplate(template=template, input_variables=["question"])

In [6]:
prompt.format(question="Can Barack Obama have a conversation with George Washington?")

"Question: Can Barack Obama have a conversation with George Washington?\n\nLet's think step by step.\n\nAnswer: "

In [9]:
llm(prompt)

TypeError: Object of type PromptTemplate is not JSON serializable

## 3. Chains

Combine LLMs and Prompts in multi-step workflows

In [10]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "Can Barack Obama have a conversation with George Washington?"

print(llm_chain.run(question))

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


 No, it is not possible for Barack Obama to have a conversation with George Washington because George Washington is no longer alive.


## 4. Agents and Tools

Agents involve an LLM making decisions about which Actions to take, taking that Action, seeing an Observation, and repeating that until done.


When used correctly agents can be extremely powerful. In order to load agents, you should understand the following concepts:

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.
- LLM: The language model powering the agent.
- Agent: The agent to use.

Tools: https://python.langchain.com/en/latest/modules/agents/tools.html

Agent Types: https://python.langchain.com/en/latest/modules/agents/agents/agent_types.html

In [11]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [12]:
!pip install wikipedia

  Using cached beautifulsoup4-4.12.2-py3-none-any.whl (142 kB)
  Using cached soupsieve-2.4.1-py3-none-any.whl (36 kB)
Using legacy 'setup.py install' for wikipedia, since package 'wheel' is not installed.
    Running setup.py install for wikipedia ... done
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [13]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0)
tools = load_tools(["wikipedia", "llm-math"], llm=llm)

In [14]:
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [15]:
agent.run("In what year was the film Departed with Leopnardo Dicaprio released? What is this year raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out the year the film was released and then use the calculator to calculate the power.
Action: Wikipedia
Action Input: Departed with Leonardo Dicaprio
Observation: Page: Leonardo DiCaprio filmography
Summary: Leonardo DiCaprio is an American actor who began his career performing as a child on television. He appeared on the shows The New Lassie (1989) and Santa Barbara (1990) and also had long running roles in the comedy-drama Parenthood (1990) and the sitcom Growing Pains (1991). DiCaprio played Tobias "Toby" Wolff opposite Robert De Niro in the biographical coming-of-age drama This Boy's Life in 1993. In the same year, he had a supporting role as a developmentally disabled boy Arnie Grape in What's Eating Gilbert Grape, which earned him nominations for the Academy Award for Best Supporting Actor and the Golden Globe Award for Best Supporting Actor – Motion Picture. In 1995, DiCaprio played the leading roles of an American author 

'The film Departed with Leonardo DiCaprio was released in 2006 and 2006 raised to the 0.43 power is 26.30281917656938.'

## 5. Memory

Add State to Chains and Agents.

Memory is the concept of persisting state between calls of a chain/agent. LangChain provides a standard interface for memory, a collection of memory implementations, and examples of chains/agents that use memory.

In [16]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.predict(input="Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [17]:
conversation.predict(input="Can we talk about AI?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI?
AI:

> Finished chain.


' Absolutely! What would you like to know about AI?'

In [18]:
conversation.predict(input="I'm interested in Reinforcement Learning.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: Can we talk about AI?
AI:  Absolutely! What would you like to know about AI?
Human: I'm interested in Reinforcement Learning.
AI:

> Finished chain.


' Sure! Reinforcement Learning is a type of machine learning algorithm that allows an AI agent to learn from its environment by taking actions and receiving rewards or punishments. It is used to solve complex problems that require trial and error. Would you like to know more about how it works?'

## 6. Document Loaders

Combining language models with your own text data is a powerful way to differentiate them. The first step in doing this is to load the data into “documents” - a fancy way of say some pieces of text. This module is aimed at making this easy.

https://python.langchain.com/en/latest/modules/indexes/document_loaders.html

In [19]:
from langchain.document_loaders import NotionDirectoryLoader

loader = NotionDirectoryLoader("Notion_DB")

docs = loader.load()

## 7. Indexes

Indexes refer to ways to structure documents so that LLMs can best interact with them. This module contains utility functions for working with documents

- Embeddings: An embedding is a numerical representation of a piece of information, for example, text, documents, images, audio, etc.
- Text Splitters: When you want to deal with long pieces of text, it is necessary to split up that text into chunks.
- Vectorstores: Vector databases store and index vector embeddings from NLP models to understand the meaning and context of strings of text, sentences, and whole documents for more accurate and relevant search results.

In [20]:
import requests

url = "https://raw.githubusercontent.com/hwchase17/langchain/master/docs/modules/state_of_the_union.txt"
res = requests.get(url)
with open("state_of_the_union.txt", "w") as f:
  f.write(res.text)

In [21]:
# Document Loader
from langchain.document_loaders import TextLoader
loader = TextLoader('./state_of_the_union.txt')
documents = loader.load()

In [22]:
# Text Splitter
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [23]:
!pip install sentence_transformers

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [24]:
# Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

#text = "This is a test document."
#query_result = embeddings.embed_query(text)
#doc_result = embeddings.embed_documents([text])

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
!pip install faiss-cpu

     |████████████████████████████████| 2.7 MB 3.4 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.9/bin/python3 -m pip install --upgrade pip' command.


In [26]:
# Vectorstore: https://python.langchain.com/en/latest/modules/indexes/vectorstores.html
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embeddings)

query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [27]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


In [28]:
db.save_local("faiss_index")
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search(query)
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## End-to-end example

https://github.com/hwchase17/chat-langchain
